<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>

# Laboratorio Limpieza de Datos

En este laboratorio usaremos el DataFrame de Netflix completo creado en los primeros laboratorios de Pandas. 

**Instrucciones:**

1. Lee cuidadosamente el enunciado de cada ejercicio.

2. Implementa la solución en la celda de código proporcionada.

3. Documenta todas las funciones creadas durante el ejercicio. 

4. Debes incluir después de cada gráfica la interpretación de las mismas en una celda de markdown. 

In [2]:
import pandas as pd
import numpy as np

import itertools
from pandas import ExcelWriter
from datetime import datetime

pd.set_option("display.max_columns", None)
df_contenido = pd.read_csv("../datos/df_contenido.csv", index_col=0)

## Parte 1: Limpieza y Preparación de Datos

#### Ejercicio 1: Estandarización y limpieza de columnas

En este ejercicio, debes limpiar y estandarizar algunas columnas clave para hacerlas más manejables y consistentes en tus análisis. Específicamente, trabajarás con las columnas `date_added` y `duration` para convertirlas a un formato uniforme y estructurado.

Instrucciones:

1. **Convertir la columna `date_added`**: La columna `date_added` contiene fechas en formato de texto. Debes convertirla a un formato `datetime` que pandas pueda entender y manejar fácilmente.

2. **Limpiar la columna `duration`**: La columna `duration` tiene valores en diferentes formatos como "1 Season", "2 Seasons", "90 min", etc. Tu tarea es extraer el número (ya sea el número de temporadas o la cantidad de minutos) y crear una nueva columna llamada `duration_cleaned` con esos valores estandarizados.


**Resultado Esperado:**
Deberás obtener algo como esto:

| duration   | duration_cleaned |
|------------|-----------------|
| 1 Season   | 1               |
| 90 min     | 90              |
| 2 Seasons  | 2               |
| 45 min     | 45              |
| 3 Seasons  | 3               |

### Conversion columna date_added

In [3]:
df_contenido["date_added_modif"] = pd.to_datetime(df_contenido["date_added"],format='mixed')

### Limpieza columna duration

In [4]:
df_contenido['duration_clean'] = df_contenido['duration'].str.lower().str.replace('seasons', '').str.replace(' min', '')

In [5]:
df_contenido['duration_clean'].unique()

array(['90', '2 ', nan, '125', '9 ', '104', '127', '161', '61', '166',
       '147', '111', '110', '124', '116', '98', '23', '122', '85', '83',
       '113', '13', '182', '48', '145', '80', '117', '128', '119', '143',
       '118', '63', '121', '142', '154', '120', '82', '229', '76', '156',
       '112', '107', '129', '135', '136', '165', '150', '133', '70', '84',
       '140', '78', '7 ', '64', '59', '139', '69', '148', '189', '141',
       '130', '138', '81', '132', '10 ', '123', '68', '66', '62', '74',
       '131', '39', '46', '38', '8 ', '17 ', '126', '155', '159', '137',
       '12', '273', '36', '34', '77', '60', '49', '58', '204', '212',
       '25', '73', '29', '47', '32', '35', '71', '149', '33', '15', '54',
       '224', '162', '37', '75', '79', '55', '158', '164', '173', '181',
       '185', '21', '51', '42', '22', '177', '13 ', '52', '14', '53', '8',
       '28', '50', '9', '26', '45', '171', '27', '44', '146', '20', '157',
       '17', '203', '41', '30', '194', '15 ', '23

#### Ejercicio 2: Normalización de la columna `rating`

La columna `rating` tiene diferentes calificaciones como `PG`, `PG-13`, `R`, entre otras. Debes categorizar estas calificaciones en tres grupos:

- **'General Audience'** para calificaciones como `G`, `PG`.

- **'Teens'** para calificaciones como `PG-13`, `TV-14`.

- **'Adults'** para calificaciones como `R`, `TV-MA`.


In [6]:
diccionario_mapeo = {
                    "G" : "General_Audience",
                    "PG" : "General_Audience",
                    "TV-PG" : "General_Audience",
                    "TV-Y" : "General_Audience",
                    "TV-Y7" : "General_Audience",
                    "TV-Y7-FV" : "General_Audience",                    
                    "TV-G" : "General_Audience",
                    "TV-Y7" : "General_Audience",
                    "PG-13": "Teens",
                    "TV-14": "Teens",
                    "R": "Adults",
                    "TV-MA": "Adults",
                    "NC-17": "Adults"}

df_contenido["rating_normal"] = df_contenido["rating"].map(diccionario_mapeo)

# Filtrar filas con valores no válidos en "rating_normal"
invalid_ratings = df_contenido[df_contenido["rating_normal"].isna()]

# Crear una lista de valores válidos
lista_valida = ['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R','TV-G', 'G', 'NC-17','NR','TV-Y7-FV', 'UR']

# Asignar "Unknown" a valores que no son válidos
df_contenido.loc[~df_contenido["rating"].isin(lista_valida), "rating_normal"] = "Unknown"



df_contenido["rating_normal"].unique()

array(['Teens', 'Adults', 'General_Audience', 'Unknown', nan],
      dtype=object)

#### Ejercicio 3: Creación de una columna personalizada basada en el elenco

Vamos a identificar si un actor clave como `Leonardo DiCaprio`, `Tom Hanks`, o `Morgan Freeman` aparece en el elenco.

Usa `apply` y una función lambda para crear una nueva columna llamada `has_famous_actor` que contenga `True` si alguno de estos actores está en la lista de `cast` y `False` en caso contrario.

In [7]:
df_contenido.sample()

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Es_original,date_added_modif,duration_clean,rating_normal
show_id,,,,,,,,,,,,,,,,,,,,
s1110,Movie,Just Say Yes,"Appie Boudellah, Aram van de Rest","Yolanthe Cabau, Noortje Herlaar, Kim-Lian van ...",Netherlands,"April 2, 2021",2021,TV-MA,98 min,"Comedies, International Movies, Romantic Movies",Incurable romantic Lotte finds her life upende...,Romantic comedy,"April 2, 2021",97.0,4.5,Dutch,1.0,2021-04-02,98,Adults


In [8]:
actor_clave = ["Leonardo DiCaprio", "Tom Hanks","Morgan Freeman"]

df_contenido["has_famous_actor"] = df_contenido["cast"].apply(lambda x: any(actor in x for actor in actor_clave) if isinstance(x,str) else False)

df_contenido.groupby("has_famous_actor")["title"].count().reset_index()


,has_famous_actor,title
0,False,8772
1,True,35


#### Ejercicio 4: Creación de una columna personalizada usando lógica condicional

Vamos a crear una columna llamada `is_recent` que identifique si un título fue lanzado en los últimos 5 años.

Crea una función para marcar con `True` si el título es reciente (lanzado en los últimos 5 años) y `False` si no lo es.

In [9]:
df_contenido["is_recent"] = df_contenido["release_year"].apply(lambda x: True if pd.notnull(x) and (datetime.now().year - x <= 5) else False)


df_contenido[["release_year", "is_recent"]].sample(20)

,release_year,is_recent
show_id,,
s611,2003,False
s815,1981,False
s1948,2020,True
s2524,2019,True
s5230,2017,False
s5457,2017,False
s5249,2016,False
s416,2009,False
s2969,2019,True


#### Ejercicio 5: Clasificación de películas por década

En este ejercicio, tu objetivo es categorizar los años de lanzamiento de las películas o series en décadas. La columna `release_year` contiene el año de lanzamiento y debes crear una nueva columna llamada `decade` que indique la década correspondiente, como "1990s", "2000s", etc.


In [10]:
df_contenido["release_year"].sort_values() #primero vemos desde donde comenzarian las decadas

show_id
s4251    1925
s7791    1942
s8206    1942
s8740    1943
s8764    1943
         ... 
s4       2021
s3       2021
s19      2021
s2       2021
s69      2021
Name: release_year, Length: 8807, dtype: int64

In [11]:
años = [1920, 1930, 1940, 1960, 1970, 1980, 1990, 2000, 2010, 2020, 2030]
decadas = ["1920s","1930s", "1940s", "1960s", "1970s", "1980s", "1990s", "2000s", "2010s", "2020s"]

df_contenido["decade"] = pd.cut(df_contenido["release_year"], bins=años, labels=decadas,include_lowest=False)


df_contenido[["release_year", "decade"]].sample(20)

,release_year,decade
show_id,,
s5600,2016,2010s
s1051,2021,2020s
s4656,2018,2010s
s7831,2005,2000s
s1869,2020,2010s
s1580,2020,2010s
s6432,1958,1940s
s2372,2019,2010s
s6044,2018,2010s


#### Ejercicio 6: Extracción de información

Para practicar la extracción de información:

1. **Extrae el primer actor** de la lista en la columna `cast` y crea una nueva columna llamada `first_actor`.

2. **Extrae el primer nombre del director** y guárdalo en una columna llamada `first_name_director`.


In [12]:
df_contenido["cast_strg"] = df_contenido["cast"].astype(str)
df_contenido["first_actor"] = df_contenido["cast_strg"].apply(lambda x: x.split(",")[0] if x != "nan" else None)

df_contenido["director_strg"] = df_contenido["director"].astype(str)
df_contenido["first_director"] = df_contenido["director_strg"].apply(lambda x: x.split(",")[0] if x != "nan" else None)

df_contenido[["cast_strg","first_actor","director_strg","first_director"]].sample(10)

,cast_strg,first_actor,director_strg,first_director
show_id,,,,
s1162,nan,None,nan,None
s7359,Luciana Aymar,Luciana Aymar,Ana Quiroga,Ana Quiroga
s1153,"Tara Sands, Anairis Quiñones, Laura Stahl, Jen...",Tara Sands,nan,None
s1015,"Jacob Ewaniuk, Kyle Breitkopf, Addison Holley",Jacob Ewaniuk,nan,None
s448,"Kristen Stewart, Robert Pattinson, Taylor Laut...",Kristen Stewart,Bill Condon,Bill Condon
s263,nan,None,Laura Brownson,Laura Brownson
s417,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jig...",Vatsal Dubey,Rajiv Chilaka,Rajiv Chilaka
s3574,David Attenborough,David Attenborough,nan,None
s4744,"Hugo Becker, Wilfred Benaïche, Christophe Kour...",Hugo Becker,nan,None


#### Ejercicio 7: Limpieza de la columna `cast`

La columna `cast` contiene una lista de actores separados por comas. Tu objetivo es realizar las siguientes tareas:

1. **Reemplaza los valores nulos** en la columna `cast` por "sin información".

2. **Contar el número de actores** en cada entrada y crear una nueva columna llamada `num_cast`.

3. **Normalizar los nombres**: Asegúrate de que los nombres de los actores estén en un formato consistente (por ejemplo, quitar espacios adicionales).

In [13]:
df_contenido["cast"]=df_contenido["cast"].fillna("sin informacion") #remplazamos valores nulos

df_contenido["num_cast"] = df_contenido["cast"].apply(lambda x: len(x.split(",")) if x!= "sin informacion" else 0) #contamos n° de actires

df_contenido["cast"] = df_contenido["cast"].apply(lambda x: ", ".join([name.strip() for name in x.split(",")]) if x != "sin información" else x)#normalizamos columna

df_contenido[["cast","num_cast"] ]

,cast,num_cast
show_id,,
s1,sin informacion,0
s2,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",19
s3,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",9
s4,sin informacion,0
s5,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",8
...,...,...
s8803,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",10
s8804,sin informacion,0
s8805,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",7



#### Ejercicio 9: Identificación de Directores Recurrentes

En este ejercicio, debes identificar los directores que aparecen más de una vez en el conjunto de datos. Realiza los siguientes pasos:

1. **Reemplaza los valores nulos** en la columna `director` por "sin información".

3. **Cuenta cuántas veces aparece cada director** en la columna creada en el ejercicio 6.

4. **Filtra aquellos directores que aparecen más de una vez** y crea una nueva columna llamada `recurrent_director` donde se indique "Yes" si el director aparece varias veces o "No" en caso contrario.

In [14]:
df_contenido["first_director"] =df_contenido["first_director"].fillna("sin informacion")#remplazamos nulos


In [15]:

directores_frecuencia = df_contenido ["first_director"].value_counts() #contamos valores de directores
directores_frecuencia_df = directores_frecuencia.reset_index()
directores_frecuencia_df

,first_director,count
0,sin informacion,2634
1,Rajiv Chilaka,22
2,Raúl Campos,18
3,Suhas Kadav,16
4,Marcus Raboy,16
...,...,...
4401,Antoni Krauze,1
4402,Jeremiah Zagar,1
4403,David Serrano,1
4404,Max Joseph,1


In [16]:
df_contenido["recurrent_director"]=df_contenido["first_director"].apply(lambda x: "Yes" if directores_frecuencia.get(x, 0) > 1 and x != "sin informacion" else "No")

df_contenido[["first_director", "recurrent_director"]]

,first_director,recurrent_director
show_id,,
s1,Kirsten Johnson,No
s2,sin informacion,No
s3,Julien Leclercq,Yes
s4,sin informacion,No
s5,sin informacion,No
...,...,...
s8803,David Fincher,Yes
s8804,sin informacion,No
s8805,Ruben Fleischer,Yes
